In [ ]:
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt

from livecellx.core.datasets import LiveCellImageDataset
from livecellx.sample_data import tutorial_three_image_sys

from livecellx.trajectory.feature_extractors import compute_haralick_features, compute_skimage_regionprops
from livecellx.preprocess.utils import normalize_img_to_uint8
from livecellx.core.parallel import parallelize
from livecellx.core.io_sc import prep_scs_from_mask_dataset
from livecellx.core.single_cell import SingleCellTrajectoryCollection, create_sc_table
import livecellx.core.single_cell
from livecellx.core.single_cell import SingleCellStatic
from livecellx.core.io_utils import LiveCellEncoder



## Load datasets

load CTC or CX-A data

In [ ]:

# # dataset_dir_path = Path(
# #     "../datasets/celltrackingchallenge/BF-C2DL-HSC/BF-C2DL-HSC/01"
# # )


# dataset_dir_path = Path(
#     # r"../datasets/cxa-high-density/contrast_enhanced_factor=3_png_data/C10^3_wellC7_RI_MIP_stitched",
#      r"../datasets/cxa-high-density/contrast_enhanced_factor=3_png_data/C0.5^4_wellE6_RI_MIP_stitched",
# )

# # mask_dataset_path = Path("../datasets/celltrackingchallenge/DIC-C2DH-HeLa/DIC-C2DH-HeLa/01_GT/TRA")
# # mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="tif")
# def extract_mask_time(filename):
#     """seg_{time}.png -> time"""
#     return int(filename.split("_")[1].split(".")[0])

# # mask_dataset_dir = Path(r"./application_results/celltrackingchallenge/dic-hela_segmentation_cellpose_tn1/seg")
# # mask_time2url_paths = sorted(glob.glob(str((Path(mask_dataset_dir) / Path("*.png")))))
# # mask_time2url = {extract_mask_time(Path(path).name): path for path in mask_time2url_paths}
# # mask_dataset = LiveCellImageDataset(time2url=mask_time2url, ext="png")

# time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*.png")))))
# time2url = {i: path for i, path in enumerate(time2url)}
# img_dataset = LiveCellImageDataset(time2url=time2url, ext="png")



load EBSS 120hrs data XY09

In [ ]:
# EBSS_120hrs_XY09_folder = r"E:\xing-imaging\tifs_CFP_A549_VIM_120hr_NoTreat_NA_YL_Ti2e_2023-03-22\tifs_CFP_A549_VIM_120hr_NoTreat_NA_YL_Ti2e_2023-03-22\XY09\DIC"

# # filename sample: CFP_A549_VIM_120hr_NoTreat_NA_YL_Ti2e_2023-03-22 _T0001_XY09_DIC.tif
# EBSS_120hrs_XY09_time2url = sorted(glob.glob(str((Path(EBSS_120hrs_XY09_folder) / Path("*.tif")))))
# img_dataset = LiveCellImageDataset(time2url={i: path for i, path in enumerate(EBSS_120hrs_XY09_time2url)}, ext="tif")

In [ ]:
# list(img_dataset.time2url.items())[:5]

In [ ]:
# len(img_dataset), img_dataset.time2url

In [ ]:
# for time in sc_img_dataset.time2url:
#     print("sc:", sc_img_dataset.time2url[time])
#     print("img:", img_dataset.time2url[time])

In [ ]:
# annotation_dir = Path("../datasets/mitosis-annotations-2023/shiman_CXA_normal/C10^3")
# annotation_dir = Path("../datasets/mitosis-annotations-2023/shiman_CXA_normal/C0.5^4")
# annotation_dir = Path("../datasets/mitosis-annotations-2023/shiman_CXA_normal/C0.5^4_wellE6_RI_MIP_stitched")
annotation_dir = Path("../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4")
# annotation_dir = Path(r"../datasets/mitosis-annotations-2023/shiman_XY09/XY09")
# annotation_dir = Path(r"../datasets/mitosis-annotations-2023/shiman_XY10/XY10")
# classes = ["normal"]
classes = ["mitosis"]

In [ ]:
from livecellx.track.classify_utils import load_all_json_dirs


class2samples, class2samples_extra_info = load_all_json_dirs([annotation_dir], class_subfolders=classes)


In [ ]:
all_scs = SingleCellStatic.load_single_cells_json(annotation_dir/"single_cells.json")

Compute the features  
Read the features in the next section if you already computed the features.

In [ ]:
# from livecellx.core.io_sc import prep_scs_from_mask_dataset
# scs = prep_scs_from_mask_dataset(mask_dataset, img_dataset)

annotated_scs = []
for cls in class2samples:
    for samples in class2samples[cls]:
        annotated_scs.extend(samples)

In [ ]:
from typing import List
from livecellx.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)

sc_img_dataset = annotated_scs[0].img_dataset
sdata = track_SORT_bbox_from_scs(annotated_scs, sc_img_dataset, max_age=0, min_hits=1)

In [ ]:
# sct_sample_dict = {}
# for cls in class2samples:
#     sct_sample_dict[cls] = []
#     for i, sample in enumerate(class2samples[cls]):
#         if len(sample) == 0:
#             continue
#         assert '_annotation_label_info' in sample[0].meta, "sample[0].meta should have '_annotation_label_info'"
#         annotation_label_infos = sample[0].meta['_annotation_label_info']
#         annotation_label_info = None
#         for info in annotation_label_infos:
#             if info['label'] == cls:
#                 annotation_label_info = info
#                 break
#         assert annotation_label_info is not None, "sc meta shoud contain annotation_label_info == cls: {}".format(cls)
#         sample_id = annotation_label_info['sample_id']
#         sample_dict = {"sample": sample, "sample_id": sample_id}
#         sct_sample_dict[cls].append(sample_dict)


In [ ]:
all_scs[0].meta

Set the labels of sc

In [ ]:
for sc in annotated_scs:
    for i in range(len(sc.meta["_annotation_label_info"])):
        sc.meta["_annotation_label_info"][i]["label"] = "someLabel"

In [ ]:
from livecellx.core.single_cell import (
    SingleCellTrajectoryCollection,
    SingleCellStatic,
    SingleCellTrajectory,
    create_sctc_from_scs,
    filter_sctc_by_time_span,
)

from livecellx.core.sct_operator import create_sctc_edit_viewer_by_interval, create_scs_edit_viewer_by_interval
tmp_sctc = create_sctc_from_scs(annotated_scs)
# tmp_sctc = create_sctc_from_scs(all_scs)
# sct_operator = create_sctc_edit_viewer_by_interval(tmp_sctc, img_dataset=img_dataset, span_interval=1500, contour_sample_num=15)

In [ ]:
import importlib
import livecellx
import livecellx.core.sct_operator
importlib.reload(livecellx.core.sct_operator)
sc_img_dataset = annotated_scs[0].img_dataset
# sct_operator = livecellx.core.sct_operator.create_scs_edit_viewer_by_interval(annotated_scs, img_dataset=sc_img_dataset, span_interval=1500, contour_sample_num=50)
sct_operator = livecellx.core.sct_operator.create_sctc_edit_viewer_by_interval(sdata, img_dataset=sc_img_dataset, span_interval=1500, contour_sample_num=50)

load the annotations

In [ ]:
sct_operator.load_annotations([annotation_dir], classes=classes)

In [ ]:
len(sc_img_dataset)

In [ ]:
sc_annotation_labels = []
sc_annotation_labels_strs = []
for sc in sct_operator.shape_layer.properties["sc"]:
    sc_annotation_labels.append([])
    sc_annotation_labels_strs.append("")
    if "_annotation_label_info" not in sc.meta:
        continue
    for info in sc.meta["_annotation_label_info"]:
        sc_annotation_labels[-1].extend([info["label"]])
    sc_annotation_labels[-1] = list(set(sc_annotation_labels[-1]))
    annotation_repr = ",".join(sc_annotation_labels[-1])
    sc_annotation_labels_strs[-1] = annotation_repr

all_properties = sct_operator.shape_layer.properties
all_properties["annotation_labels"] = sc_annotation_labels_strs
sct_operator.shape_layer.properties = all_properties
sct_operator.shape_layer.text = "annotation_labels"

In [ ]:
sct_operator.shape_layer.properties.keys()

In [ ]:
sct_operator.shape_layer.properties.keys()

In [ ]:
# # out_dir = Path("./tmp/DIC-C2DH-HeLa/01_annotation")
# out_dir = Path("./tmp/fixed_annotations")
# out_dir.mkdir(exist_ok=True, parents=True)
# sct_operator.save_annotations(out_dir)

In [ ]:
# from livecellx.core.sc_seg_operator import create_sc_seg_napari_ui
# from livecellx.core.sct_operator import create_sctc_edit_viewer_by_interval, create_scs_edit_viewer_by_interval
# from livecellx.core.single_cell import (
#     SingleCellTrajectoryCollection,
#     SingleCellStatic,
#     SingleCellTrajectory,
#     create_sctc_from_scs,
#     filter_sctc_by_time_span,
# )

# import importlib
# importlib.reload(livecellx.core.sct_operator)

# tmp_sdata = create_sctc_from_scs(sdata.get_all_scs())
# sct_operator = livecellx.core.sct_operator.create_sctc_edit_viewer_by_interval(tmp_sdata, img_dataset, span_interval=1000)

Set annotation labels

In [ ]:
sc_annotation_labels = []
sc_annotation_labels_strs = []
for sc in sct_operator.shape_layer.properties["sc"]:
    sc_annotation_labels.append([])
    sc_annotation_labels_strs.append("")
    if "_annotation_label_info" not in sc.meta:
        continue
    for info in sc.meta["_annotation_label_info"]:
        sc_annotation_labels[-1].extend([info["label"]])
    sc_annotation_labels[-1] = list(set(sc_annotation_labels[-1]))
    annotation_repr = ",".join(sc_annotation_labels[-1])
    sc_annotation_labels_strs[-1] = annotation_repr

all_properties = sct_operator.shape_layer.properties
all_properties["annotation_labels"] = sc_annotation_labels_strs
sct_operator.shape_layer.properties = all_properties
sct_operator.shape_layer.text = "annotation_labels"

In [ ]:
# from livecellx.core.io_sc import prep_scs_from_mask_dataset
# scs = prep_scs_from_mask_dataset(mask_dataset, img_dataset)

In [ ]:
# from livecellx.track.movie import generate_single_trajectory_movie

# for track_id, traj in traj_collection:
#     generate_single_trajectory_movie(traj, save_path= out_dir / f"track_{track_id}.gif")

In [ ]:
sample_traj = sdata[19]
sample_traj.show_on_grid(padding=200, ax_height=4, ax_width=4, )

In [ ]:

sample_traj.show_on_grid(padding=200, ax_height=3, ax_width=3, dims=[400, 400], dims_offset=[30, 30], )